# Preparation of the drinking water quality data

In [65]:
# Imports
from PIL import Image
import pandas as pd
import numpy as np
import os,sys

In [66]:
# Read raw files
raw_path = "./raw_data/terviseamet"
raw_files = os.listdir(raw_path)

original_columns = pd.read_csv(os.path.join(raw_path, raw_files[0]))

raw_filelist = []
for f in raw_files:
    df = pd.read_csv(os.path.join(raw_path, f))

    # select columns and translate into english
    col_map = {
        'id': 'test_id',
        'veevark_id': 'station_id', 
        'veeliik': 'water_type', 
        'proovivotu_aeg': 'date',
        'proovivotu_metoodika': 'test_method', 
        'id3': 'indicator_id',
        'nimetus4': 'indicator_name',
        'sisaldus': 'value',
        'yhik': 'unit',
        'hinnang5': 'assessment'
        }
    df = df[col_map.keys()]
    df.rename(columns=col_map, inplace=True)
    
    # Add year variable
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df.date.dt.year

    # Add df to list for concatenation later
    raw_filelist.append(df)

long_df = pd.concat(raw_filelist, ignore_index=True)
long_df

/tmp/ipykernel_212/3289904645.py:9: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(raw_path, f))
/tmp/ipykernel_212/3289904645.py:9: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(raw_path, f))
/tmp/ipykernel_212/3289904645.py:9: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(raw_path, f))
/tmp/ipykernel_212/3289904645.py:9: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(raw_path, f))


,test_id,station_id,water_type,date,test_method,indicator_id,indicator_name,value,unit,assessment,year
0,276444,302,Joogivesi,2022-05-07,NaN,241,Maitse (lahjendusaste),1,lahjendusaste,vastab nõuetele,2022
1,276444,302,Joogivesi,2022-05-07,NaN,112,Lõhn (lahjendusaste),1,lahjendusaste,vastab nõuetele,2022
2,276444,302,Joogivesi,2022-05-07,NaN,131,Värvus (Pt/Co skaala),3,mg/l Pt,vastab nõuetele,2022
3,276444,302,Joogivesi,2022-05-07,NaN,16,pH,7.7,pH ühik,vastab nõuetele,2022
4,276444,302,Joogivesi,2022-05-07,NaN,51,Elektrijuhtivus,630,μS/cm,vastab nõuetele,2022
...,...,...,...,...,...,...,...,...,...,...,...
504185,173704,306,Joogivesi,2018-01-01,NaN,131,Värvus (Pt/Co skaala),3,mg/l Pt,vastab nõuetele,2018
504186,173704,306,Joogivesi,2018-01-01,NaN,16,pH,7.22,pH ühik,vastab nõuetele,2018
504187,173878,306,Joogivesi,2018-01-01,NaN,7,Clostridium perfringens (koos eostega),0,PMÜ/100ml,vastab nõuetele,2018
504188,173878,306,Joogivesi,2018-01-01,NaN,4,Escherichia coli,0,PMÜ/100 ml,vastab nõuetele,2018


In [67]:
long_df.groupby(by=['year','assessment']).size()

year  assessment                    
2014  Halvem kui kvaliteediklass III        1
      Kvaliteediklass I                    41
      ei vasta nõuetele                   690
      vastab nõuetele                   69970
2015  Kvaliteediklass I                    10
      ei vasta nõuetele                   573
      vastab nõuetele                   71163
2016  Kvaliteediklass I                     4
      ei vasta nõuetele                   593
      vastab nõuetele                   72709
2017  ei vasta nõuetele                   607
      vastab nõuetele                   71393
2018  ei vasta nõuetele                   443
      vastab nõuetele                   52280
2019  ei vasta nõuetele                   352
      vastab nõuetele                   33635
2020  ei vasta nõuetele                   386
      vastab nõuetele                   56834
2021  ei vasta nõuetele                   468
      vastab nõuetele                   58411
2022  ei vasta nõuetele                    

In [68]:
# How many tests per year per station_id
long_df.groupby(by=['station_id', 'year']).test_id.unique()

station_id  year
162         2014                                [80238, 80290, 61616]
            2015                              [106003, 106041, 98823]
            2016                             [133003, 131795, 122653]
            2017                                     [137400, 137538]
            2018    [182057, 182240, 177174, 177336, 168987, 16925...
                                          ...                        
2604        2022                                     [268222, 268532]
2643        2022                                             [274479]
2676        2022                                     [274169, 274168]
2678        2021                             [256134, 256136, 256138]
2679        2022                                             [275717]
Name: test_id, Length: 9237, dtype: object

In [69]:
long_df.value.dtype

dtype('O')

# What indicators to keep

In [70]:
# Keep only 2017-2021
period_df = long_df[(long_df.year>=2019) & (long_df.year < 2022)]

In [277]:
# Which indicators exist for all years
inds_per_year_df = period_df.groupby(by=['indicator_name', 'year']).size().sort_values(ascending=False).to_frame(name='size')
inds_per_year_wide = inds_per_year_df.pivot_table(
    index = ['indicator_name'],
    columns = 'year',
    values = 'size'
)

# Indicators that have more than n instances for all years
inds_that_exist_all_years = inds_per_year_wide.dropna()
for y in period_df.year.unique():
    inds_that_exist_all_years = inds_that_exist_all_years[inds_that_exist_all_years[y]>=500]
indicators_to_keep = inds_that_exist_all_years.index
indicators_to_keep

Index(['Ammoonium', 'Coli-laadsed bakterid',
       'Coli-laadsed bakterid (Colilert)', 'Elektrijuhtivus', 'Enterokokid',
       'Escherichia coli', 'Escherichia coli (Colilert)', 'Hägusus (NTU)',
       'Kolooniate arv 22 °C', 'Lõhn (lahjendusaste)', 'Lõhn (pallides)',
       'Maitse (lahjendusaste)', 'Maitse (pallides)', 'Mangaan', 'Raud',
       'Värvus (Pt/Co skaala)', 'pH'],
      dtype='object', name='indicator_name')

In [278]:
# Dataframe with only top indicators
rows_to_keep = [iname in indicators_to_keep for iname in period_df.indicator_name]
top_ind_df = period_df[rows_to_keep] 
top_ind_df

,test_id,station_id,water_type,date,test_method,indicator_id,indicator_name,value,unit,assessment,year
13627,212401,1724,Joogivesi,2019-12-31 00:00:00,NaN,40,Kolooniate arv 22 °C,87,PMÜ/1 ml,vastab nõuetele,2019
13628,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",112,Lõhn (lahjendusaste),1,lahjendusaste,vastab nõuetele,2019
13629,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",56,Hägusus (NTU),1.0,NTU,vastab nõuetele,2019
13630,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",1,Raud,18,µg/l,vastab nõuetele,2019
13632,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",131,Värvus (Pt/Co skaala),2,mg/l Pt,vastab nõuetele,2019
...,...,...,...,...,...,...,...,...,...,...,...
379714,247588,306,Joogivesi,2021-03-01 00:00:00,NaN,736,Coli-laadsed bakterid (Colilert),0,MPN/100 ml,vastab nõuetele,2021
379716,247587,306,Joogivesi,2021-02-01 00:00:00,NaN,736,Coli-laadsed bakterid (Colilert),0,MPN/100 ml,vastab nõuetele,2021
379717,247587,306,Joogivesi,2021-02-01 00:00:00,NaN,737,Escherichia coli (Colilert),0,MPN/100 ml,vastab nõuetele,2021
379718,247586,306,Joogivesi,2021-01-01 00:00:00,NaN,737,Escherichia coli (Colilert),0,MPN/100 ml,vastab nõuetele,2021


## Tests and stations that 

In [279]:
# Which tests in 2021 didn't pass the evaluation
test_compliance_df = top_ind_df[top_ind_df.year == 2021][['test_id', 'station_id', 'assessment']].assign(count=1).pivot_table(
    values = 'count',
    index = ['test_id', 'station_id'],
    columns = 'assessment',
    aggfunc = 'sum'
)

test_compliance_tmp = ['non-compliant' if val >= 1 else 'compliant' for val in test_compliance_df['ei vasta nõuetele']]
test_compliance_df['compliance'] = test_compliance_tmp
test_compliance_df.reset_index(inplace=True)

# If any test for station is non-compliant then non compliant
compliance_df = test_compliance_df.assign(count=1).pivot_table(
    values = 'count',
    index = ['station_id'],
    columns = 'compliance',
    aggfunc = 'sum'
)
compliance_df
compliance_tmp = ['non-compliant' if val >= 1 else 'compliant' for val in compliance_df['non-compliant']]
compliance_df['compliance'] = compliance_tmp
compliance_df.reset_index(inplace=True)
compliance_df.groupby(by='compliance').size()


compliance
compliant        936
non-compliant    173
dtype: int64

In [280]:
# Pivot table for test & indicator
single_test_wide = top_ind_df[['test_id', 'indicator_name', 'value']].pivot_table(
    index = 'test_id',
    columns = 'indicator_name',
    values = 'value'
)

/tmp/ipykernel_212/1162552586.py:2: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  single_test_wide = top_ind_df[['test_id', 'indicator_name', 'value']].pivot_table(


In [281]:
# How many measurements per station per year
ind_by_year = top_ind_df.groupby(by=['indicator_name', 'year']).size()

In [282]:
# Value column to numeric and errors to NaN
top_ind_df = top_ind_df.assign(value = pd.to_numeric(top_ind_df['value'], errors='coerce'))
top_ind_df

,test_id,station_id,water_type,date,test_method,indicator_id,indicator_name,value,unit,assessment,year
13627,212401,1724,Joogivesi,2019-12-31 00:00:00,NaN,40,Kolooniate arv 22 °C,87.0,PMÜ/1 ml,vastab nõuetele,2019
13628,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",112,Lõhn (lahjendusaste),1.0,lahjendusaste,vastab nõuetele,2019
13629,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",56,Hägusus (NTU),1.0,NTU,vastab nõuetele,2019
13630,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",1,Raud,18.0,µg/l,vastab nõuetele,2019
13632,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",131,Värvus (Pt/Co skaala),2.0,mg/l Pt,vastab nõuetele,2019
...,...,...,...,...,...,...,...,...,...,...,...
379714,247588,306,Joogivesi,2021-03-01 00:00:00,NaN,736,Coli-laadsed bakterid (Colilert),0.0,MPN/100 ml,vastab nõuetele,2021
379716,247587,306,Joogivesi,2021-02-01 00:00:00,NaN,736,Coli-laadsed bakterid (Colilert),0.0,MPN/100 ml,vastab nõuetele,2021
379717,247587,306,Joogivesi,2021-02-01 00:00:00,NaN,737,Escherichia coli (Colilert),0.0,MPN/100 ml,vastab nõuetele,2021
379718,247586,306,Joogivesi,2021-01-01 00:00:00,NaN,737,Escherichia coli (Colilert),0.0,MPN/100 ml,vastab nõuetele,2021


# Wide data

In [283]:
indicator_name_map = {
'Hägusus (NTU)' : 'turbidity', 
'Escherichia coli': 'escherichia_coli', 
'Coli-laadsed bakterid': 'coli-type_bacteria',
'Värvus (Pt/Co skaala)': 'color',
'Raud': 'iron',
'Maitse (lahjendusaste)': 'taste',
'Lõhn (lahjendusaste)': 'smell', 
'pH': 'ph', 
'Elektrijuhtivus': 'conductivity', 
'Ammoonium': 'ammonia', 
'Lõhn (pallides)': 'smell2', 
'Maitse (pallides)': 'taste2',
'Kolooniate arv 22 °C': 'number_of_colonies', 
'Enterokokid': 'enterococci', 
'Coli-laadsed bakterid (Colilert)': 'coli-type_bacteria_colilert', 
'Escherichia coli (Colilert)': 'escherichia-coli_colilert',
'Mangaan': 'manganese', 
'Clostridium perfringens (koos eostega)': 'clostridium_perfringens', 
'Oksüdeeritavus': 'oxidability'
}

In [284]:
# Make indicator names more readable
new_names = [indicator_name_map[n] for n in top_ind_df.indicator_name]
to_keep_df = top_ind_df.assign(indicator_name = new_names)

# Keep only 2019 and 2020 as variables
to_keep_df = to_keep_df[to_keep_df.year < 2021]

# Create new variable representing indicator and year
to_keep_df['year'] = to_keep_df.year.apply(str)
to_keep_df = to_keep_df.assign(indyear = lambda x:  x.indicator_name + "_" + x.year)

# Make variable with both year and name in it
to_keep_df

,test_id,station_id,water_type,date,test_method,indicator_id,indicator_name,value,unit,assessment,year,indyear
13627,212401,1724,Joogivesi,2019-12-31 00:00:00,NaN,40,number_of_colonies,87.00,PMÜ/1 ml,vastab nõuetele,2019,number_of_colonies_2019
13628,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",112,smell,1.00,lahjendusaste,vastab nõuetele,2019,smell_2019
13629,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",56,turbidity,1.00,NTU,vastab nõuetele,2019,turbidity_2019
13630,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",1,iron,18.00,µg/l,vastab nõuetele,2019,iron_2019
13632,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",131,color,2.00,mg/l Pt,vastab nõuetele,2019,color_2019
...,...,...,...,...,...,...,...,...,...,...,...,...
176828,219341,306,Joogivesi,2020-02-01 00:00:00,NaN,736,coli-type_bacteria_colilert,0.00,MPN/100 ml,vastab nõuetele,2020,coli-type_bacteria_colilert_2020
176829,219341,306,Joogivesi,2020-02-01 00:00:00,NaN,56,turbidity,0.38,NTU,vastab nõuetele,2020,turbidity_2020
176830,219341,306,Joogivesi,2020-02-01 00:00:00,NaN,131,color,3.00,mg/l Pt,vastab nõuetele,2020,color_2020
176831,219106,306,Joogivesi,2020-01-01 00:00:00,NaN,736,coli-type_bacteria_colilert,0.00,MPN/100 ml,vastab nõuetele,2020,coli-type_bacteria_colilert_2020


In [285]:
# Keep only relevant variables
wide_df = to_keep_df.pivot_table(
    index = ['station_id'],
    columns = 'indyear',
    values = 'value',
    aggfunc = np.nanmax
)

In [297]:
# Keep rows with max N missing values
n = int(wide_df.shape[1]/2)
missing_count = wide_df.isna().T.sum()
max_n_missing = missing_count[missing_count <= n].to_frame()

In [298]:
# Keep only rows with not more than max allowed number of missing values
clean_df = wide_df[[i in max_n_missing.index for i in wide_df.index]]
clean_df.reset_index(inplace=True)
clean_df

indyear,station_id,ammonia_2019,ammonia_2020,coli-type_bacteria_2019,coli-type_bacteria_2020,coli-type_bacteria_colilert_2019,coli-type_bacteria_colilert_2020,color_2019,color_2020,conductivity_2019,...,smell2_2019,smell2_2020,smell_2019,smell_2020,taste2_2019,taste2_2020,taste_2019,taste_2020,turbidity_2019,turbidity_2020
0,163,0.08,0.08,0.0,0.0,NaN,0.0,11.7,12.1,716.0,...,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.18,1.90
1,165,0.08,0.08,0.0,0.0,NaN,NaN,7.0,7.8,1167.0,...,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.52,0.50
2,167,0.08,0.08,0.0,0.0,NaN,NaN,7.7,9.1,996.0,...,NaN,NaN,1.0,3.0,NaN,NaN,1.0,3.0,3.90,1.54
3,169,0.29,0.23,0.0,0.0,NaN,NaN,NaN,NaN,536.0,...,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.30,0.40
4,170,0.25,0.24,0.0,0.0,NaN,NaN,8.8,7.0,631.0,...,NaN,NaN,5.0,2.0,NaN,NaN,5.0,2.0,0.62,1.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798,2302,NaN,0.13,NaN,NaN,71.4,1.0,NaN,6.0,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.00
799,2303,NaN,0.05,NaN,NaN,0.0,2.0,NaN,6.0,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,4.0,NaN,1.80
800,2305,NaN,NaN,0.0,NaN,NaN,2.0,7.0,3.8,805.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.70,0.50
801,2307,0.53,NaN,NaN,NaN,0.0,0.0,2.0,9.0,742.0,...,NaN,NaN,2.0,4.0,NaN,NaN,2.0,4.0,1.00,2.40


In [299]:
# Add results
withresults_df = clean_df.merge(compliance_df[['station_id','compliance']], on='station_id')

In [300]:
# The average value of variables based on compliance 
withresults_df.groupby('compliance').mean()

,station_id,ammonia_2019,ammonia_2020,coli-type_bacteria_2019,coli-type_bacteria_2020,coli-type_bacteria_colilert_2019,coli-type_bacteria_colilert_2020,color_2019,color_2020,conductivity_2019,...,smell2_2019,smell2_2020,smell_2019,smell_2020,taste2_2019,taste2_2020,taste_2019,taste_2020,turbidity_2019,turbidity_2020
compliance,,,,,,,,,,,,,,,,,,,,,
compliant,862.246951,0.115847,0.111287,0.767152,0.691667,14.042697,0.690323,4.022163,4.071103,561.863692,...,0.405797,0.5,1.550495,1.631373,0.420290,0.500000,1.511530,1.595876,1.217804,1.199513
non-compliant,957.009615,0.241347,0.200203,2.106061,1.228070,2.874419,2.120000,5.216000,5.369000,592.194175,...,0.500000,0.9,1.637363,2.078652,0.571429,0.666667,1.592105,1.936709,1.826569,2.205294


# Train and test data

In [312]:
# Rename compliance column
withresults_df.rename(columns={'compliance':'result2021'}, inplace=True)

# Select 70% of data for training
train_df = withresults_df.sample(n = int(withresults_df.shape[0] * 0.7))

# Test data & solution
test_df = withresults_df.loc[~withresults_df.index.isin(train_df.index)]

solution_df = test_df.copy(deep=True)
test_df.pop('result2021')

# Sample submission
result_vals = train_df['result2021'].unique()
sample_submission_df = solution_df.copy(deep=True)
sample_submission_df['result2021'] = np.random.choice(result_vals, solution_df.shape[0])

In [313]:
# Write data
result_path = "./prepped_data/terviseamet"

train_df.to_csv(os.path.join(result_path, 'train.csv'), index=False)
test_df.to_csv(os.path.join(result_path, 'test.csv'), index=False)
solution_df.to_csv(os.path.join(result_path, 'solution.csv'), index=False)
sample_submission_df.to_csv(os.path.join(result_path, 'sample_submission.csv'), index=False)

In [314]:
# The average value of variables based on compliance 
train_df.groupby('result2021').size()

result2021
compliant        452
non-compliant     80
dtype: int64

In [315]:
# The average value of variables based on compliance 
solution_df.groupby('result2021').size()

result2021
compliant        204
non-compliant     24
dtype: int64

In [303]:
# Check that sample objects doesn't match other objects
sample_submission_df['result2021'] == solution_df['result2021']

10     False
12      True
13      True
17     False
20      True
       ...  
738    False
743     True
745    False
747     True
748    False
Name: result2021, Length: 228, dtype: bool

In [304]:
# Try to train the model
import sklearn as sk

train_df.fillna(0, inplace=True)
train_result = train_df.pop('result2021')
solution_df.fillna(0, inplace=True)
solution_result = solution_df.pop('result2021')


In [305]:
from sklearn.tree import DecisionTreeClassifier

# Treenime mudeli, kasutades treenimise ettenähtud andmeid
model = DecisionTreeClassifier(max_depth=5)
model.fit(X=train_df, y=train_result)

# Teeme prognoosi testandmete peal
tree_prediction = model.predict(X=solution_df)



In [306]:
pred_results = solution_df.assign(
    actual = solution_result, 
    predicted = tree_prediction,
    correct = lambda x: x.actual == x.predicted)

accuracy = pred_results.correct.sum() / pred_results.shape[0]
accuracy

0.8421052631578947

In [309]:
from sklearn.metrics import confusion_matrix, f1_score
print(confusion_matrix(solution_result, tree_prediction))
f1_score(
    [s == 'compliant' for s in solution_result],
    [s == 'compliant' for s in tree_prediction],
)

[[190   7]
 [ 29   2]]


0.9134615384615384